## Integration tests of federated data in time series 

In [1]:
import sys
import pandas as pd
import numpy as np
sys.path.append(".../ADL_platform//S-ADL") # dir library

In [2]:
from flexanomalies.datasets.preprocessing_utils import scaling
from SADL.federated_data.algorithms import flexanomalies
from SADL.time_series.time_series_utils import TimeSeriesProcessor
from SADL.time_series.time_series_datasets_uci import global_load as load_time_series 
from sklearn.model_selection import train_test_split

2025-10-01 18:30:00.487926: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-01 18:30:00.499864: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759336200.514375   11304 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759336200.518539   11304 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-10-01 18:30:00.533941: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

##### Example of time series data set for testing and preprocessiong

In [3]:
X,y = load_time_series('ai4i_2020_predictive_maintenance_dataset')   #name dataset in static datasets uci repo
labels = y["Machine failure"]
X = X.drop('Type',axis=1)  # remove Type or apply one hot encoding

Metadata: {'uci_id': 601, 'name': 'AI4I 2020 Predictive Maintenance Dataset', 'repository_url': 'https://archive.ics.uci.edu/dataset/601/ai4i+2020+predictive+maintenance+dataset', 'data_url': 'https://archive.ics.uci.edu/static/public/601/data.csv', 'abstract': 'The AI4I 2020 Predictive Maintenance Dataset is a synthetic dataset that reflects real predictive maintenance data encountered in industry.', 'area': 'Computer Science', 'tasks': ['Classification', 'Regression', 'Causal-Discovery'], 'characteristics': ['Multivariate', 'Time-Series'], 'num_instances': 10000, 'num_features': 6, 'feature_types': ['Real'], 'demographics': [], 'target_col': ['Machine failure', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF'], 'index_col': ['UID', 'Product ID'], 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2020, 'last_updated': 'Wed Feb 14 2024', 'dataset_doi': '10.24432/C5HS5C', 'creators': [], 'intro_paper': {'ID': 386, 'type': 'NATIVE', 'title': 'Explainable Artificial 

In [ ]:
# from SADL.static_data.preprocessing.preprocessing_static import OneHotEncoderPreprocessing
# encoder = OneHotEncoderPreprocessing(columns=['Type'])
# data_encoded = encoder.fit_transform(X)

In [4]:
from SADL.time_series.preprocessing.preprocessing_ts import StandardScalerPreprocessing 
scaler = StandardScalerPreprocessing()
X_scaled = scaler.fit_transform(X)


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, labels, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape )

(8000, 5) (2000, 5) (8000,) (2000,)


#### Autoencoder Model for anomaly detection in time series 

In [6]:
kwargs = {
    "algorithm_": "autoencoder",
    "contamination":0.1,
    "label_parser": None,
    "epochs": 3,
    "input_dim": X.shape[1],
    "batch_size": 16,
    "neurons": [16, 8, 16],
    "hidden_act": ["relu", "relu", "relu"],
    "preprocess":False,
    "w_size": 10,
    "n_pred": 1,
     "n_clients":3,
     "n_rounds":5,
     }



modelAE = flexanomalies.FlexAnomalyDetection(**kwargs)
print(modelAE.get_params())


Federated Params:{'n_clients': 3, 'n_rounds': 5} 
 Model Params:{'algorithm_': 'autoencoder', 'contamination': 0.1, 'label_parser': None, 'epochs': 3, 'input_dim': 5, 'batch_size': 16, 'neurons': [16, 8, 16], 'hidden_act': ['relu', 'relu', 'relu'], 'preprocess': False, 'w_size': 10, 'n_pred': 1}
Inspecting model's attributes:
contamination: 0.1
input_dim: 5
neurons: [16, 8, 16]
model_path: 
w_size: 10
n_pred: 1
callbacks: [<keras.src.callbacks.early_stopping.EarlyStopping object at 0x7fead9eac520>]
hidden_act: ['relu', 'relu', 'relu']
output_act: linear
loss: mse
validation_size: 0.2
batch_size: 16
epochs: 3
optimizer: adam
preprocess: False
scaler: StandardScaler()
model: <Sequential name=sequential_1, built=True>
algorithm_: autoencoder
label_parser: None
{'label_parser': None, 'algorithm_': 'autoencoder', 'contamination': 0.1, 'input_dim': 5, 'neurons': [16, 8, 16], 'model_path': '', 'w_size': 10, 'n_pred': 1, 'callbacks': [<keras.src.callbacks.early_stopping.EarlyStopping object at

/data/Beatriz/Doctorado GR/ADL_platform/S-ADL/venv/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-10-01 18:30:07.569240: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [7]:
processor = TimeSeriesProcessor(window_size= kwargs["w_size"], step_size=1, future_prediction=False)
X_train_windows, y_train_windows, X_test_windows, y_test_windows = processor.process_train_test(X_train, y_train, X_test, y_test)
print("X_train shape:", X_train_windows.shape)
print("y_train shape:", y_train_windows.shape)
print("X_test shape:", X_test_windows.shape)
print("y_test shape:", y_test_windows.shape)

X_train shape: (7991, 10, 5)
y_train shape: (7991, 10)
X_test shape: (1991, 10, 5)
y_test shape: (1991, 10)


In [8]:
modelAE.fit(X_train_windows,y_train_windows)



Running round: 0

Training model at client.
Epoch 1/3


/data/Beatriz/Doctorado GR/ADL_platform/S-ADL/venv/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 18 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


120/120 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.8495 - val_loss: 0.2879
Epoch 2/3
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2139 - val_loss: 0.0677
Epoch 3/3
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0599 - val_loss: 0.0451
Training model at client.
Epoch 1/3
120/120 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.8292 - val_loss: 0.2797
Epoch 2/3
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2049 - val_loss: 0.0678
Epoch 3/3
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0605 - val_loss: 0.0458
Training model at client.
Epoch 1/3
120/120 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.8061 - val_loss: 0.2784
Epoch 2/3
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2039 - val_loss: 0.0716
Epoch 3/3
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0612 - val_loss: 0.0464

Running round: 1

Training model at client.
Epoch 1/3
120/120 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0425 - val_loss: 0.0307
Epoch 2/3
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.02

In [9]:
scores_pred = modelAE.decision_function(X_train_windows)
print("Scores",scores_pred)
modelAE.predict(X_train_windows)

250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 870us/step
Scores [[0.09133731 0.07052949 0.07441299 0.08728958 0.08234749]
 [0.09136265 0.07042566 0.07392014 0.08705194 0.08239626]
 [0.11177074 0.0925007  0.07637783 0.08882729 0.08276979]
 ...
 [0.03351913 0.0514194  0.0626675  0.06088672 0.06657641]
 [0.02890762 0.05260877 0.07354184 0.0724305  0.06324413]
 [0.02844625 0.03784797 0.07436368 0.07142065 0.06136212]]
Inspecting model's attributes:
contamination: 0.1
input_dim: 5
neurons: [16, 8, 16]
model_path: 
w_size: 10
n_pred: 1
callbacks: [<keras.src.callbacks.early_stopping.EarlyStopping object at 0x7fead9eac520>]
hidden_act: ['relu', 'relu', 'relu']
output_act: linear
loss: mse
validation_size: 0.2
batch_size: 16
epochs: 3
optimizer: adam
preprocess: False
scaler: StandardScaler()
model: <Sequential name=sequential_1, built=True>
algorithm_: autoencoder
label_parser: None
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 743us/step


array([[[-0.85064334, -0.61488485,  0.4218949 , -0.8981446 ,
          1.3647168 ],
        [-0.9252213 , -1.0704498 , -0.8397806 ,  1.3876759 ,
          0.43864793],
        [-0.86843413, -1.4910406 , -0.07116989, -0.9070854 ,
          1.3839048 ],
        ...,
        [ 0.3268913 ,  1.1654321 ,  0.68619794, -0.7383018 ,
         -1.4480834 ],
        [ 0.8491761 ,  0.5992035 ,  0.37977678, -0.68882066,
          0.8157907 ],
        [-0.7814479 , -1.1568537 ,  0.69748527, -1.2247394 ,
         -1.482001  ]],

       [[-0.9252213 , -1.0704498 , -0.8397806 ,  1.3876759 ,
          0.43864793],
        [-0.86843413, -1.4910406 , -0.07116989, -0.9070854 ,
          1.3839048 ],
        [ 0.43277466,  0.5579653 ,  0.30750465, -0.68473023,
         -1.6156231 ],
        ...,
        [ 0.8491761 ,  0.5992035 ,  0.37977678, -0.68882066,
          0.8157907 ],
        [-0.7814479 , -1.1568537 ,  0.69748527, -1.2247394 ,
         -1.482001  ],
        [ 0.7502518 ,  0.7352373 , -0.44266492, 

In [10]:
modelAE.model.d_scores_ ==scores_pred
modelAE.model.labels_


array([1., 1., 1., ..., 0., 0., 0.])

In [11]:
modelAE.model.labels_

array([1., 1., 1., ..., 0., 0., 0.])

In [ ]:
modelAE.evaluate(X_test_windows, X_test_windows, label_test = y_test_windows.flatten().astype("int"))

In [ ]:
from SADL.visualization_module import DataVisualizationScoresTS
visualizer = DataVisualizationScoresTS(scores_pred.ravel())
fig = visualizer.visualize(method="percentile", threshold=0.95)  # percentil
fig.show()

#### CNN_LSTM model for anomaly forcasting  in time series 

In [6]:
kwargs = {
    "algorithm_": "deepCNN_LSTM",
    "contamination":0.1,
    "label_parser": None,
    "epochs": 3,
    "input_dim": X.shape[1],
    "batch_size": 8,
    "filters_cnn": [8, 6],
    "units_lstm": [8,6],
    "kernel_size": [4,4],
    "hidden_act": ["relu", "relu"],
    "w_size": 20,
    "n_pred": 1,
     "n_clients":2,
     "n_rounds":3,
     }



modelDeep = flexanomalies.FlexAnomalyDetection(**kwargs)


Federated Params:{'n_clients': 2, 'n_rounds': 3} 
 Model Params:{'algorithm_': 'deepCNN_LSTM', 'contamination': 0.1, 'label_parser': None, 'epochs': 3, 'input_dim': 5, 'batch_size': 8, 'filters_cnn': [8, 6], 'units_lstm': [8, 6], 'kernel_size': [4, 4], 'hidden_act': ['relu', 'relu'], 'w_size': 20, 'n_pred': 1}
relu 6 4


/data/Beatriz/Doctorado GR/ADL_platform/S-ADL/venv/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-10-01 18:27:13.564659: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 17, 8)          │           168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 8, 8)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 5, 6)           │           198 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 2, 6)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 2, 8)           │           480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 2, 6)           │           360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 12)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 5)              │            65 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 1, 5)           │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,271 (4.96 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_2 (Conv1D)               │ (None, 17, 8)          │           168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 8, 8)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 5, 6)           │           198 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_3 (MaxPooling1D)  │ (None, 2, 6)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 2, 8)           │           480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 2, 6)           │           360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 12)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │            65 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_1 (Reshape)             │ (None, 1, 5)           │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,271 (4.96 KB)

 Trainable params: 1,271 (4.96 KB)

 Non-trainable params: 0 (0.00 B)

None


In [7]:
processor =   TimeSeriesProcessor(window_size= kwargs["w_size"], step_size=1, future_prediction=True, n_pred=kwargs["n_pred"])
X_train_windows, y_train_windows, X_test_windows, y_test_windows, l_test_windows = processor.process_train_test(X_train, y_train, X_test, y_test,l_test=y_test)
print("X_train shape:", X_train_windows.shape)
print("y_train shape:", y_train_windows.shape)
print("X_test shape:", X_test_windows.shape)
print("y_test shape:", y_test_windows.shape)
print("l_test shape:",l_test_windows.shape)

X_train shape: (7980, 20, 5)
y_train shape: (7980, 1, 5)
X_test shape: (1980, 20, 5)
y_test shape: (1980, 1, 5)
l_test shape: (1980, 1)


In [8]:
modelDeep.fit(X_train_windows,y_train_windows)



Running round: 0

Training model at client.
Epoch 1/3


/data/Beatriz/Doctorado GR/ADL_platform/S-ADL/venv/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 26 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


359/359 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 1.0432 - val_loss: 0.9845
Epoch 2/3
359/359 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.0368 - val_loss: 0.9856
Epoch 3/3
359/359 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.0545 - val_loss: 0.9859
Training model at client.
Epoch 1/3
359/359 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.9733 - val_loss: 1.0227
Epoch 2/3
359/359 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.9527 - val_loss: 1.0224
Epoch 3/3
359/359 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.9881 - val_loss: 1.0238

Running round: 1

Training model at client.
Epoch 1/3
359/359 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 1.0314 - val_loss: 0.9847
Epoch 2/3
359/359 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.0242 - val_loss: 0.9853
Epoch 3/3
359/359 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.0223 - val_loss: 0.9860
Training model at client.
Epoch 1/3
359/359 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.9417 - val_loss: 1.0221
Epoch 2/3
359/359 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.97

In [9]:
pred = modelDeep.predict(X_test_windows,y_test_windows)
pred

Inspecting model's attributes:
contamination: 0.1
input_dim: 5
filters_cnn: [8, 6]
units_lstm: [8, 6]
kernel_size: [4, 4]
w_size: 20
n_pred: 1
callbacks: [<keras.src.callbacks.early_stopping.EarlyStopping object at 0x7014683e8ee0>]
model_path: 
hidden_act: ['relu', 'relu']
output_act: linear
loss: mse
validation_size: 0.2
batch_size: 8
epochs: 3
optimizer: adam
model: <Sequential name=sequential_1, built=True>
algorithm_: deepCNN_LSTM
label_parser: None


62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


array([[[-0.07114381, -0.02937591, -0.03508598,  0.01735242,
          0.01043208]],

       [[-0.21164015, -0.17198423, -0.06246889,  0.0828605 ,
          0.0209099 ]],

       [[-0.05639213, -0.02088488, -0.01505328,  0.00858508,
          0.01417448]],

       ...,

       [[-0.08984806, -0.07452107, -0.0420606 ,  0.0135102 ,
         -0.01904198]],

       [[-0.07422794, -0.08654504, -0.00520325,  0.02067835,
         -0.02108238]],

       [[-0.05444936, -0.06481352, -0.00599966,  0.00164375,
         -0.02491266]]], dtype=float32)

In [10]:
modelDeep.model.labels_

array([0, 0, 1, ..., 1, 1, 1])

In [12]:
l_test_windows.shape

(1980, 1)

In [11]:
scores_pred = modelDeep.decision_function(X_test_windows,y_test_windows)

62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [20]:
modelDeep.evaluate(X_test_windows, y_test_windows, label_test = l_test_windows.flatten().astype("int"))

62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
Acc: 51.970% 

Precision: 0.048 

F1score: 0.090 

Recall: 0.797 

AUC_ROC: 0.654 



In [ ]:
from SADL.visualization_module import DataVisualizationScoresTS
visualizer = DataVisualizationScoresTS(scores_pred.ravel())
fig = visualizer.visualize(method="percentile", threshold=0.95)  # percentil
fig.show()